In [1]:
# Change width of Jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Section 1: Formatting the initial dataset
I downloaded CSVs from 77 different Industry designations from CrunchBase. In this first section, I use glob to read in every file from the data folder and then concatenate them into a single, large dataframe.

In [2]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
# %matplotlib inline
import glob 


path = r'old/Data' # use your path
all_files = glob.glob(path + "/*.csv")
# using a more balanced collection of companies...i downloaded companies based on category type
# in an attempt to level the imbalanced dataset 

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
type(df)
# df.to_csv('feb1_frame.csv')

pandas.core.frame.DataFrame

There are 61,671 rows in this dataframe and 12 different columns.

In [3]:
df.shape

(61671, 12)

### Get rid of all the duplicates in the master_dataframe by using the Company's URL as the subset

In [4]:
# Get rid of all the duplicates in the master_dataframe by using the CB URL as the subset
df = df.drop_duplicates(subset='Organization Name URL').reset_index(drop=True)
df.shape
# (47204, 13) now there are 47,204 companies, which means that almost 10k companies were duplicated and thus dropped. 

(47204, 12)

There are 47,204 unique companies in this dataframe and 12 different columns.

In [5]:
df.columns

Index(['Organization Name', 'Organization Name URL', 'Website', 'Industries',
       'Industry Groups', 'Description', 'Full Description', 'LinkedIn',
       'Hub Tags', 'Twitter', 'Facebook', 'CB Rank (Organization)'],
      dtype='object')

The four columns we are going to care most about are "Organization Name", "Industries", "Industry Groups", and "Full Description". I am using "Full Description" instead of the shorter "Description" so that I have more text to work with in our NLP model.

In [6]:
df = df[['Organization Name', 'Industries', 'Industry Groups', 'Full Description']]

df.head()

,Organization Name,Industries,Industry Groups,Full Description
0,SimilarWeb,"Analytics, Artificial Intelligence, Business I...","Artificial Intelligence, Data and Analytics, D...",SimilarWeb offers an AI-based market intellige...
1,SurveyMonkey,"Analytics, Human Resources, Internet, Market R...","Administrative Services, Data and Analytics, D...",is a survey company which powers curious organ...
2,UserTesting,"Consumer Research, Market Research, Marketing ...","Data and Analytics, Design, Information Techno...",UserTesting is an on-demand human insights pla...
3,Sensor Tower,"Analytics, Android, Big Data, iOS, Market Rese...","Apps, Data and Analytics, Design, Mobile, Plat...",Sensor Tower offers solution for mobile market...
4,Numerator,"Analytics, Brand Marketing, Market Research","Data and Analytics, Design, Sales and Marketing",Numerator is a market intelligence firm that b...


#### Now I'm going to save this dataframe down to 1_df.csv and then read it back in

In [7]:
df.to_csv("1_df.csv")

# Section 2: Preprocessing 
Next order of business: I need to create a dataframe of companies I am 100% certain are of a certain industry; this will be my test dataset. This is somewhat tricky and I'm going to make a few assumptions here, which isn't perfect, but it's good enough to get started.

The CrunchBase system assigns companies Industries and Industry Groups, some have one or two tags, some have more than 12. My approach for building out a train set is going to be to use companies that got only a single industry designation from CrunchBase. 

In [8]:
# Then I will parse the Industries and Industry Groups columns so that it is a list of terms
# This function also creates our test set of companies that have only a single Industry designation

# for loop to extract companies that have less than 2 industries assigned
# to their CB Industries column:
def split_industries(dataframe, column):

    counter = 0
    industry_split = []
    #loop through each company in the CB Industries column
    for industry in dataframe[column]:
        #turn company to string
        industry = str(industry)
        #if there are only one listed industry for the company
        
        #use regex to split on both ',' and ';'
        import re
        split_inds = re.split('; |,', industry)
        
        if split_inds[0]!='nan':
            industry_split.append(split_inds)
            
        #just using a counter because i'm curious, can be removed
        if len(split_inds) > 2:
            counter+=1
    
    return industry_split, counter

#### First I'll parse the Industries column, then I'll parse the Industry Groups column

In [9]:
# Industries column
df_industries_split, num_counter = split_industries(df, 'Industries')

# Now assign the df_industry_split to the df['Industries']
df['Industries'] = df_industries_split

########################

# Industry Groups column
df_industry_groups_split, num_counter = split_industries(df, 'Industry Groups')

# Now assign the df_industry_split to the df['Industries']
df['Industry Groups'] = df_industry_groups_split

#### Companies with 1 Industry Group from CrunchBase

In [10]:
#https://stackoverflow.com/questions/45089650/filter-dataframe-rows-based-on-length-of-column-values

df_reduced = df[(df['Industry Groups'].str.len() == 1)]
df_reduced.sample(5)

,Organization Name,Industries,Industry Groups,Full Description
41482,Upperline Health,"[Health Care, Hospital, Personal Health]",[Health Care],Upperline Health has a variety of services to ...
4051,JustLegal,"[Legal, Legal Tech]",[Professional Services],JustLegal offers a simple and secure technolog...
45561,Cathcart Rail,"[Railroad, Transportation]",[Transportation],Cathcart Rail is a diversified freight rail pl...
44306,Greenline Trade,"[Logistics, Public Transportation, Shipping,...",[Transportation],GreenLine Trade is an international freight fo...
23411,Save-A-Lot,[Food Processing],[Food and Beverage],Save-A-Lot stores bring discount groceries to ...


In [11]:
# how many companies with only a single 
len(df_reduced)

3197

#### Companies with 2 Industry Groups from CrunchBase

In [12]:
df_two = df[(df['Industry Groups'].str.len() == 2)]

In [13]:
# how many companies with two industry designations from crunchbase 
len(df_two)

8834

I'm also going to take the first industry designation for companies that were given two industries from CrunchBase. Again, this isn't perfect, but I manually read through about a hundred of these companies and the first of the pair made plenty of sense for a classification.

In [14]:
df_two.loc[:,"Industry Groups"] = [[ind[0]] for ind in df_two["Industry Groups"]]

/Users/holdenbruce/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


#### Now merge the two, appending df_two to df_reduced

In [15]:
df_reduced = df_reduced.append(df_two)
df_reduced.sample(10)

,Organization Name,Industries,Industry Groups,Full Description
20787,Kynectiv,"[E-Learning, Training]",[Education],DecisionSim is a simulation-based learning pla...
32243,ObjectWin Technology,"[Recruiting, Staffing Agency]",[Administrative Services],"ObjectWin Technology, Inc. is a leading inform..."
29669,Mable,"[E-Commerce, Wholesale]",[Commerce and Shopping],Mable is a mobile and web-based B2B wholesale ...
32302,Medical Employment Directory,"[Health Care, Staffing Agency]",[Administrative Services],"Address: 2343 Weldon Pkwy, St Louis, MO 63146,..."
21759,Lascaux Co.,"[Internet, Mobile, Social Network]",[Internet Services],Lascaux Co. is focused on making art fun again...
30426,Brixey & Meyer,"[Accounting, Consulting, Financial Services]",[Financial Services],Brixey & Meyer is a registered accounting firm...
36697,ERI Economic Research Institute,"[Analytics, Human Resources]",[Administrative Services],"The ERI Economic Research Institute, Inc., is ..."
32456,Science Exchange,"[Consulting, Outsourcing]",[Professional Services],Science Exchange is the world’s leading R&D Se...
12432,Morris Bank,"[Banking, Finance, Financial Services]",[Financial Services],Morris Bank operates as a state-chartered comm...
15202,AA Auto Protection,"[Automotive, Financial Services, Insurance, ...",[Financial Services],AA Auto Protection is insurance backed and one...


#### Convert list of Industry Groups and Industries columns back to string

In [16]:
# https://stackoverflow.com/questions/37347725/converting-a-panda-df-list-into-a-string/37347844
df_reduced.loc[:,'Industries'] = df_reduced.loc[:,'Industries'].apply(', '.join)
df_reduced.loc[:,'Industry Groups'] = df_reduced.loc[:,'Industry Groups'].apply(', '.join)

#### How many unique Industry Groups are there in our df_reduced dataset?

In [17]:
df_reduced['Industry Groups'].unique()

array(['Manufacturing', 'Privacy and Security', 'Information Technology',
       'Professional Services', 'Software', 'Education', 'Health Care',
       'Travel and Tourism', 'Food and Beverage', 'Transportation',
       'Clothing and Apparel', 'Real Estate', 'Administrative Services',
       'Sales and Marketing', 'Internet Services', 'Financial Services',
       'Commerce and Shopping', 'Energy', 'Sports', 'Other', 'Hardware',
       'Data and Analytics', 'Design', 'Consumer Goods',
       'Agriculture and Farming', 'Government and Military', 'Mobile',
       'Media and Entertainment', 'Community and Lifestyle', 'Gaming',
       'Apps', 'Sustainability', 'Science and Engineering',
       'Natural Resources', 'Advertising', 'Content and Publishing',
       'Biotechnology', 'Navigation and Mapping', 'Consumer Electronics',
       'Events'], dtype=object)

In [18]:
len(df_reduced['Industry Groups'].unique())

40

There are 40 different Industry Groups represented in our training set df_reduced.

#### How many unique Industries are there in our df_reduced dataset?

In [19]:
len(df_reduced['Industries'].unique())

4398

There are 4,398 different Industries in the dataset. A lot of this is due to overlapping and unparsed data, there's probably much fewer than that. But it still goes to show that **the Industry Groups is more general than the Industries column**.

Looking at all of the unique industries that were extracted from companies with only a single industry classification from CrunchBase, it's obvious that there are some overlapping industry classifications that might make more sense to just cluster together at the outset of this project. 

It will likely take a bit of work to sort through all of these by hand and cluster them but it might be worth it. Maybe not. I don't know. For now, I'll leave it as is and keep on going.

In [21]:
len(df_reduced)

12031

In [20]:
df_reduced.to_csv('2_df.csv')